# Part 1. Data Wrangling/Feature Engineering:
### Create an analytic dataset by appropriately combining the various data files and create several new features and identify how many patients in the cohort met SIRS or sepsis criteria at some time during a hospitalization.

#### Mount to drive, import libraries and load datasets.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

In [6]:
def load_data():
    """Load data from CSV files and filter to smallest 1000 unique subject_ids with verification."""

    file_paths = {
        "vitals": "/content/drive/MyDrive/technicaltaskdata/vitals_cohort_sirs.csv",
        "labs": "/content/drive/MyDrive/technicaltaskdata/labs_cohort.csv",
        "diagnoses": "/content/drive/MyDrive/technicaltaskdata/diagnoses.csv",
        "notes": "/content/drive/MyDrive/technicaltaskdata/notes_small_cohort_v2.csv"
    }
    vitals_data = pd.read_csv(file_paths["vitals"])
    labs_data = pd.read_csv(file_paths["labs"])
    diagnoses_data = pd.read_csv(file_paths["diagnoses"])
    notes_data = pd.read_csv(file_paths["notes"])

    return vitals_data, labs_data, diagnoses_data, notes_data

vitals_data, labs_data, diagnoses_data, notes_data = load_data()


#### Merge datasets and choose specifc columns

In [7]:
def merge_and_filter_data(output_filename):

    vitals_data, labs_data, diagnoses_data, notes_data = load_data()

    dfs = [
        vitals_data.assign(source='vitals'),
        labs_data.assign(source='labs'),
        diagnoses_data.assign(source='diagnoses'),
        notes_data.assign(source='notes')
    ]


    merged_data = pd.concat(dfs, axis=0)

    all_subject_ids = pd.concat([
        vitals_data['subject_id'],
        labs_data['subject_id'],
        diagnoses_data['subject_id'],
        notes_data['subject_id']
    ]).unique()

    smallest_1000 = sorted(all_subject_ids)[:1000]

    filtered_data = merged_data[merged_data['subject_id'].isin(smallest_1000)]
    filtered_data = filtered_data.sort_values(['subject_id', 'charttime'])

    filtered_data.to_csv(output_filename, index=False)
    print(f"Saved filtered merged data with {len(filtered_data)} records from 1000 subjects to {output_filename}")

    return filtered_data

final_data = merge_and_filter_data('/content/drive/MyDrive/technicaltaskdata/filtered_merged_data.csv')

Saved filtered merged data with 948125 records from 1000 subjects to /content/drive/MyDrive/technicaltaskdata/filtered_merged_data.csv


In [8]:
df = pd.read_csv("/content/drive/MyDrive/technicaltaskdata/filtered_merged_data.csv")


In [12]:
#Select columns
columns_to_select = ['subject_id', 'charttime', 'icd9_code', 'note_text']
dataset = df[columns_to_select]
dataset.to_csv("/content/drive/MyDrive/technicaltaskdata/dataset.csv", index=False)

In [13]:
selected_data = pd.read_csv("/content/drive/MyDrive/technicaltaskdata/dataset.csv")

#### REGEX: First, use regex method to extract information from clinical notes ('note_text'). Create functions for each SIRS criteria. Continue modifying regex pattern in each function to get most accurate extraction.

In [22]:
selected_data = pd.read_csv("/content/drive/MyDrive/technicaltaskdata/dataset.csv")

import re

def check_temperature(text):
    if pd.isna(text):
        return False
    temp_pattern = re.compile(r'(?:temp(?:erature)?|t)\b[\s:=\-]*(\d+\.?\d*)\b\s?([°cf])?', flags=re.IGNORECASE)
    for match in temp_pattern.finditer(text):
        value, unit = match.groups()
        try:
            temp = float(value)
            if unit and unit.lower() == 'f':
                temp = (temp - 32) * 5/9
            elif not unit and temp > 45:
                temp = (temp - 32) * 5/9
            if temp > 38 or temp < 36:
                return True
        except ValueError:
            continue
    return False


def check_heart_rate(text):
    if pd.isna(text):
        return False
    hr_pattern = re.compile(r'(?:hr|heart[-\s]?rate|pulse)\b[\s:=\-]*(\d+)\b\s?(?:bpm|\/min)?\b', flags=re.IGNORECASE)
    for match in hr_pattern.finditer(text):
        hr_str = match.group(1)
        try:
            hr = int(hr_str)
            if hr > 90:
                return True
        except ValueError:
            continue
    return False


def check_resp_rate(text):
    if pd.isna(text):
        return False
    resp_pattern = re.compile(r'(?:rr|resp[-\s]?rate|respiratory[-\s]?rate|paco2|pa[\s-]?co2)\b[\s:=\-]*(\d+\.?\d*)\b\s?(?:min|\/min|mmhg)?\b', flags=re.IGNORECASE)
    for match in resp_pattern.finditer(text):
        value = match.group(1)
        try:
            num = float(value)
            if (match.group(0).lower().startswith(('rr','resp')) and num > 20) or (match.group(0).lower().startswith(('paco2','pa co2','pa-co2')) and num < 32):
                return True
        except ValueError:
            continue
    return False



def check_wbc(text):
    if pd.isna(text):
        return False
    wbc_pattern = re.compile(r'(?:wbc|white[-\s]?blood[-\s]?cells?)\b[\s:=\-]*(\d+\.?\d*)\b\s?(?:k|k\/mm3|\/mm3)?\b', flags=re.IGNORECASE)
    bands_pattern = re.compile(r'(?:bands?|immature[-\s]?granulocytes?)\b[\s:=\-]*(\d+\.?\d*)\b\s?(?:\%|percent)?\b', flags=re.IGNORECASE)
    for match in wbc_pattern.finditer(text):
        try:
            wbc = float(match.group(1))
            if wbc > 12 or wbc < 4:
                if 'k' in match.group(0).lower() or '/mm3' in match.group(0).lower():
                    if wbc > 12 or wbc < 4:
                        return True
                else:
                    if wbc > 12000 or wbc < 4000:
                        return True
        except ValueError:
            continue
    for match in bands_pattern.finditer(text):
        try:
            bands = float(match.group(1))
            if bands > 10:
                return True
        except ValueError:
            continue
    return False

In [23]:
def check_icd9_infection(icd9_code):
    if pd.isna(icd9_code):
        return False

    icd9_str = str(icd9_code)

    # 3-digit prefixes
    three_digit_prefixes = ['001','002','003','004','005','008', '009','010','011','012','013','014','015','016','017','018', '020','021','022','023','024','025','026','027','030','031', '032','033','034','035','036','037','038','039','040','041', '090','091','092','093','094','095','096','097','098','100', '101','102','103','104','110','111','112','114','115','116', '117','118','320','322','324','325','420','421','451','461', '462','463','464','465','481','482','485','486','494','510', '513','540','541','542','566','567','590','597','601','614', '615','616','681','682','683','686','730']
    if any(icd9_str.startswith(prefix) for prefix in three_digit_prefixes):
        return True

    # 4-digit prefixes
    four_digit_prefixes = ['5695','5720','5721','5750','5990','7110', '7907','9966','9985','9993']
    if any(icd9_str.startswith(prefix) for prefix in four_digit_prefixes):
        return True

    # 5-digit prefixes
    five_digit_prefixes = ['49121','56201','56203','56211','56213', '56983']
    if any(icd9_str.startswith(prefix) for prefix in five_digit_prefixes):
        return True

    return False

# Create a new column
selected_data['icd9_infection'] = selected_data['icd9_code'].apply(check_icd9_infection)

In [24]:
def check_sepsis(note_text):
    if pd.isna(note_text):
        return False
    return bool(re.search(r'\bseps[ei]s\w*|\bsept[ei]c\w*', note_text, re.IGNORECASE))

selected_data['sepsis_infection'] = selected_data['note_text'].apply(check_sepsis)

In [26]:
#SIRS Infection
selected_data['temperature'] = selected_data['note_text'].apply(check_temperature)
selected_data['heart_rate'] = selected_data['note_text'].apply(check_heart_rate)
selected_data['respiratory_rate'] = selected_data['note_text'].apply(check_resp_rate)
selected_data['wbc'] = selected_data['note_text'].apply(check_wbc)
selected_data['sirs_criteria_met'] = (
    selected_data['temperature'].astype(int) +
    selected_data['heart_rate'].astype(int) +
    selected_data['respiratory_rate'].astype(int) +
    selected_data['wbc'].astype(int)
)

sirs_patients = selected_data[selected_data['sirs_criteria_met'] >= 2]['subject_id'].unique()


#SEPSIS Infection
selected_data['infection_criteria_met'] = (
    selected_data['icd9_infection'] | selected_data['sepsis_infection']
)

sepsis_patients = selected_data[
    (selected_data['sirs_criteria_met'] >= 2) &
    (selected_data['infection_criteria_met'])
]['subject_id'].unique()


print(f"Number of patients who met SIRS criteria (REGEX): {len(sirs_patients)}")
print(f"Number of patients who met Sepsis criteria (REGEX): {len(sepsis_patients)}")

Number of patients who met SIRS criteria (REGEX): 594
Number of patients who met Sepsis criteria (REGEX): 138


#### NER: Check to see if results of extraction improve with the addition of a NER model. Use en_core_sci_lg model from scispaCy to extract information from clinical notes ('note_text') along with regex method.


##### Install scispacy and NER (en_core_sci_lg) model.

In [26]:
!pip install scispacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.

In [3]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_sci_sm: filename=en_core_sci_sm-0.5.1-py3-none-any.whl size=15870849 sha256=3dd1c41144533dcfcc2b0e0ca8edbac9a1d0ca7874c0fe363ebbf52ccd28b892
  Stored in directory: /root/.cache/pip/wheels/f0/4d/eb/0d4f64bca5fb19915b27acb2aaab5391404b0f76092d41d96d
Successfully built en_core_sci_sm


In [1]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.3/532.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.4/917.4 kB 41.5 MB/s eta 0:00:00
  Created wheel for en_core_sci_lg: filename=en_core_sci_lg-0.5.1-py3-none-any.whl size=532571809 sha256=886090489d069f58f9f47fd3879c64ff738b0e6927f1918f6a6f4e7488ba928d
  Stored in directory: /root/.cache/pip/wheels/35/7e/2a/bd14c7055451a56a21b9b0b4e9b6012d9ce299b49b6b2aecad
Successfully built en_core_sci_lg
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.3
    Uninstalling w

##### Load NER model.

In [4]:
import spacy
#nlp = spacy.load("en_core_sci_lg") #tried lg model, results came back with 0 SIR and Sepsis diagnoses
nlp = spacy.load("en_core_sci_sm")

##### Apply NER model.

In [5]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

selected_data = pd.read_csv("/content/drive/MyDrive/technicaltaskdata/dataset.csv")

def extract_ner_entities(note_text):
    if pd.isna(note_text):
        return []
    doc = nlp(str(note_text))
    return [(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in doc.ents]

selected_data['ner_entities'] = selected_data['note_text'].progress_apply(extract_ner_entities)

display(selected_data.head())

100%|██████████| 948125/948125 [58:37<00:00, 269.56it/s]


,subject_id,charttime,icd9_code,note_text,ner_entities
0,3,2101-10-20 16:40:00,NaN,NaN,[]
1,3,2101-10-20 16:40:00,NaN,NaN,[]
2,3,2101-10-20 16:40:00,NaN,NaN,[]
3,3,2101-10-20 16:40:00,NaN,NaN,[]
4,3,2101-10-20 16:40:00,NaN,NaN,[]


In [7]:
# Save the processed dataset
output_path = "/content/drive/MyDrive/technicaltaskdata/dataset_with_ner.csv"
selected_data.to_csv(output_path, index=False)

print(f"Dataset with NER entities saved to: {output_path}")
display(selected_data.head())

Dataset with NER entities saved to: /content//drive/MyDrive/technicaltaskdata/dataset_with_ner.csv


,subject_id,charttime,icd9_code,note_text,ner_entities
0,3,2101-10-20 16:40:00,NaN,NaN,[]
1,3,2101-10-20 16:40:00,NaN,NaN,[]
2,3,2101-10-20 16:40:00,NaN,NaN,[]
3,3,2101-10-20 16:40:00,NaN,NaN,[]
4,3,2101-10-20 16:40:00,NaN,NaN,[]


##### Combine both NER and Regex methods in each function created for each criteria.

In [8]:
import re

def check_temperature_ner(ner_entities):
    if not ner_entities:
        return False
    temp_pattern = re.compile(r'(?:temp(?:erature)?|t)\b[\s:=\-]*(\d+\.?\d*)\b\s?([°cf])?', flags=re.IGNORECASE)
    for entity, label, _, _ in ner_entities:
        if label in ["SIGN_SYMPTOM", "TEST"]:
            for match in temp_pattern.finditer(entity):
                value, unit = match.groups()
                try:
                    temp = float(value)
                    if unit and unit.lower() == 'f':
                        temp = (temp - 32) * 5/9
                    if temp > 38 or temp < 36:
                        return True
                except ValueError:
                    continue
    return False

def check_heart_rate_ner(ner_entities):
    if not ner_entities:
        return False
    hr_pattern = re.compile(r'(?:hr|heart[-\s]?rate|pulse)\b[\s:=\-]*(\d+)\b\s?(?:bpm|\/min)?\b', flags=re.IGNORECASE)
    for entity, label, _, _ in ner_entities:
        if label in ["SIGN_SYMPTOM", "TEST"]:
            for match in hr_pattern.finditer(entity):
                hr_str = match.group(1)
                try:
                    hr = int(hr_str)
                    if hr > 90:
                        return True
                except ValueError:
                    continue
    return False

def check_resp_rate_ner(ner_entities):
    if not ner_entities:
        return False
    resp_pattern = re.compile(r'(?:rr|resp[-\s]?rate|respiratory[-\s]?rate|paco2|pa[\s-]?co2)\b[\s:=\-]*(\d+\.?\d*)\b\s?(?:min|\/min|mmhg)?\b', flags=re.IGNORECASE)
    for entity, label, _, _ in ner_entities:
        if label in ["SIGN_SYMPTOM", "TEST"]:
            for match in resp_pattern.finditer(entity):
                value = match.group(1)
                try:
                    num = float(value)
                    if (match.group(0).lower().startswith(('rr','resp')) and num > 20) or (match.group(0).lower().startswith(('paco2','pa co2','pa-co2')) and num < 32):
                        return True
                except ValueError:
                    continue
    return False

def check_wbc_ner(ner_entities):
    if not ner_entities:
        return False
    wbc_pattern = re.compile(r'(?:wbc|white[-\s]?blood[-\s]?cells?)\b[\s:=\-]*(\d+\.?\d*)\b\s?(?:k|k\/mm3|\/mm3)?\b', flags=re.IGNORECASE)
    bands_pattern = re.compile(r'(?:bands?|immature[-\s]?granulocytes?)\b[\s:=\-]*(\d+\.?\d*)\b\s?(?:\%|percent)?\b', flags=re.IGNORECASE)
    for entity, label, _, _ in ner_entities:
        if label in ["LAB_VALUE", "TEST"]:
            for match in wbc_pattern.finditer(entity):
                try:
                    wbc = float(match.group(1))
                    if ('k' in match.group(0).lower() or '/mm3' in match.group(0).lower()) and (wbc > 12 or wbc < 4):
                        return True
                    elif wbc > 12000 or wbc < 4000:
                        return True
                except ValueError:
                    continue

            for match in bands_pattern.finditer(entity):
                try:
                    bands = float(match.group(1))
                    if bands > 10:
                        return True
                except ValueError:
                    continue
    return False

In [10]:
selected_data['temperature_ner'] = selected_data['ner_entities'].apply(check_temperature_ner)
selected_data['heart_rate_ner'] = selected_data['ner_entities'].apply(check_heart_rate_ner)
selected_data['respiratory_rate_ner'] = selected_data['ner_entities'].apply(check_resp_rate_ner)
selected_data['wbc_ner'] = selected_data['ner_entities'].apply(check_wbc_ner)

display(selected_data.head())

,subject_id,charttime,icd9_code,note_text,ner_entities,temperature_ner,heart_rate_ner,respiratory_rate_ner,wbc_ner
0,3,2101-10-20 16:40:00,NaN,NaN,[],False,False,False,False
1,3,2101-10-20 16:40:00,NaN,NaN,[],False,False,False,False
2,3,2101-10-20 16:40:00,NaN,NaN,[],False,False,False,False
3,3,2101-10-20 16:40:00,NaN,NaN,[],False,False,False,False
4,3,2101-10-20 16:40:00,NaN,NaN,[],False,False,False,False


In [11]:
def check_icd9_infection(icd9_code):
    if pd.isna(icd9_code):
        return False

    icd9_str = str(icd9_code)

    # 3-digit prefixes
    three_digit_prefixes = ['001','002','003','004','005','008', '009','010','011','012','013','014','015','016','017','018', '020','021','022','023','024','025','026','027','030','031', '032','033','034','035','036','037','038','039','040','041', '090','091','092','093','094','095','096','097','098','100', '101','102','103','104','110','111','112','114','115','116', '117','118','320','322','324','325','420','421','451','461', '462','463','464','465','481','482','485','486','494','510', '513','540','541','542','566','567','590','597','601','614', '615','616','681','682','683','686','730']
    if any(icd9_str.startswith(prefix) for prefix in three_digit_prefixes):
        return True

    # 4-digit prefixes
    four_digit_prefixes = ['5695','5720','5721','5750','5990','7110', '7907','9966','9985','9993']
    if any(icd9_str.startswith(prefix) for prefix in four_digit_prefixes):
        return True

    # 5-digit prefixes
    five_digit_prefixes = ['49121','56201','56203','56211','56213', '56983']
    if any(icd9_str.startswith(prefix) for prefix in five_digit_prefixes):
        return True

    return False


In [15]:
def check_sepsis(note_text, ner_entities):
    if pd.isna(note_text):
        return False
    if ner_entities:
        for entity, label, _, _ in ner_entities:
            if label in ["DISEASE", "CONDITION", "DIAGNOSIS"]:
                if re.search(r'\bseps[ei]s|\bsept[ei]c', entity, re.IGNORECASE):
                    return True
    return bool(re.search(r'\bseps[ei]s\w*|\bsept[ei]c\w*', note_text, re.IGNORECASE))

In [17]:
selected_data['icd9_infection'] = selected_data['icd9_code'].apply(check_icd9_infection)
selected_data['sepsis_infection'] = selected_data.apply(lambda row: check_sepsis(row['note_text'], row['ner_entities']),axis=1)

selected_data['infection_criteria_met'] = (
    selected_data['icd9_infection'] | selected_data['sepsis_infection']
)

selected_data['sirs_criteria_met_ner'] = (
    selected_data['temperature_ner'].astype(int) +
    selected_data['heart_rate_ner'].astype(int) +
    selected_data['respiratory_rate_ner'].astype(int) +
    selected_data['wbc_ner'].astype(int)
)

sirs_patients_ner = selected_data[selected_data['sirs_criteria_met_ner'] >= 2]['subject_id'].unique()

sepsis_patients_ner = selected_data[
    (selected_data['sirs_criteria_met_ner'] >= 2) &
    (selected_data['infection_criteria_met'])
]['subject_id'].unique()

print(f"Number of patients who met SIRS criteria (NER+REGEX): {len(sirs_patients_ner)}")
print(f"Number of patients who met Sepsis criteria (NER+REGEX): {len(sepsis_patients_ner)}")

Number of patients who met SIRS criteria (NER+REGEX): 0
Number of patients who met Sepsis criteria (NER+REGEX): 0


#### Create final dataset using dataset from REGEX extraction and add SIRS/SEPSIS as new features. Determine how many patients have SIR or SEPSIS.

In [29]:
# Assign 'Yes'/'No' based on SIRS/SEPSIS criteria using only the regex approach columns
selected_data['SIRS_Diagnosis'] = selected_data['sirs_criteria_met'].apply(lambda x: 'Yes' if x >= 2 else 'No')
selected_data['SEPSIS_Diagnosis'] = selected_data.apply(
    lambda row: 'Yes' if row['sirs_criteria_met'] >= 2 and (row['icd9_infection'] or row['sepsis_infection']) else 'No',
    axis=1
)

# Define the output path
output_path_final = "/content/drive/MyDrive/technicaltaskdata/final_analytic_dataset.csv"

# Export the dataframe to a CSV file
selected_data.to_csv(output_path_final, index=False)

In [30]:
# Count the number of patients with SIRS or SEPSIS diagnosis
num_sirs_patients_final = selected_data[selected_data['SIRS_Diagnosis'] == 'Yes']['subject_id'].nunique()
num_sepsis_patients_final = selected_data[selected_data['SEPSIS_Diagnosis'] == 'Yes']['subject_id'].nunique()

print(f"\nTotal number of patients with SIRS diagnosis (from final dataset): {num_sirs_patients_final}")
print(f"Total number of patients with SEPSIS diagnosis (from final dataset): {num_sepsis_patients_final}")



Total number of patients with SIRS diagnosis (from final dataset): 594
Total number of patients with SEPSIS diagnosis (from final dataset): 138


#### Explanation methods and results

**1. Data Loading and Initial Inspection:**

- Pandas DataFrames were loaded with the necessary data sets (notes_small_cohort_v2.csv, diagnoses.csv, labs_cohort.csv, and vitals_cohort_sirs.csv).
- A subset of the data was created and stored to filtered_merged_data.csv containing the smallest 1000 unique subject_id`s.
-  After loading the filtered_merged_data.csv, the dataset.csv was created by selecting particular columns (subject_id, charttime, icd9_code, note_text).

**Reasoning:** Load/review the data to comprehend the datasets' structure, content, and possible problems.  Because it drastically cuts down on processing time, creating a smaller subset of data is helpful for testing and development, particularly when working with huge datasets.  By choosing particular columns, we can make sure that we are only working with information that is most important in determining SIRS and Sepsis criteria.

**2. Initial Regex-based SIRS and Sepsis Identification:**

- **Regex for SIRS Criteria:** Regex patterns were used to generate the functions check_temperature, check_heart_rate, check_resp_rate, and check_wbc, which extract vital signs and WBC counts from the note_text column and determine if they satisfy the SIRS criteria.
- **Regex for Infection:** To determine whether the icd9_code came within predetermined ranges, a function (check_icd9_infection) was developed.  Another function (check_sepsis) searched the ote_text for the terms "sepsis" or "septic" using regex.
- **SIRS and Sepsis Criteria Aggregation:** To determine whether a patient met the SIRS criteria (at least two SIRS criteria met) or Sepsis criteria (at least two SIRS criteria met AND evidence of infection from either the ICD-9 code or the note language), the outcomes of the individual regex checks were combined.
- **Initial Results:** Using this initial regex technique, the number of distinct patients who met the SIRS and Sepsis criteria was computed and printed.

**Reasoning:** A baseline for determining SIRS and Sepsis criteria based on predetermined patterns is provided by beginning with a regex-only approach.  When the patterns are reasonably stable, this is a straightforward and frequently successful technique for obtaining particular information from text data.  While regex on clinical notes enables us to gather information that might not be formally coded, ICD-9 codes offer a systematic source of diagnostic data.

**3. Named Entity Recognition (NER) with scispaCy:**

- **Installation of scispaCy and Model:** The scispacy library and the en_core_sci_sm NER model were installed.
- **Loading NER Model:** The en_core_sci_sm model was loaded for processing.
- **Applying NER:** To apply the NER model to the note_text column and extract entities together with their labels and positions, a function called extract_ner_entities was developed.  The new column ner_entities was used to contain this data.
- **Saving Data with NER:** The DataFrame with the added ner_entities column was saved to /content/drive/MyDrive/technicaltaskdata/dataset_with_ner.csv.

**Reasoning:** More complex text analysis is made possible by the use of NER with a particular model such as scispacy.  NER offers a deeper comprehension of the text than regex alone by recognizing and classifying medical categories (such as signs, symptoms, tests, and diseases) inside the clinical notes.  This may increase the precision of finding pertinent data for SIRS and sepsis criteria.  To use this method, the library and model must be installed.

**4. Combining NER and Regex for Improved Extraction:**

- **Modified Check Functions:** The extracted NER entities were used in addition to regex in the original check functions for SIRS criteria (check_temperature_ner, check_heart_rate_ner, check_resp_rate_ner, check_wbc_ner), with a focus on particular entity labels such as "SIGN_SYMPTOM", "TEST", and "LAB_VALUE".
- **Modified Sepsis Check:** The check_sepsisfunction was modified to take into account NER items that contain the terms "sepsis" or "septic" and have names such as "DISEASE," "CONDITION," and "DIAGNOSIS."
- **SIRS and Sepsis Criteria Aggregation (NER+Regex):** The new sirs_criteria_met_ner column was utilized to evaluate if a patient met the SIRS or Sepsis criteria based on the combined NER and regex findings.
- **Comparison Results:** Using this combination approach, the number of distinct patients who met the SIRS and Sepsis criteria was computed and printed for comparison.

**Reasoning:** The goal of NER and regex combination is to take advantage of each technique's advantages.  While regex can be used to extract certain values or patterns related to medical entities, NER offers structured information about those things.  When compared to employing either strategy alone, this hybrid approach may increase the accuracy and recall of identifying the criteria.  Evaluating the efficacy of adding NER requires comparing the outcomes of the combined technique with the regex-only strategy.

**5. Final Dataset Creation and Diagnosis Assignment (Based on Regex):**

- **Diagnosis Columns:** Two new columns were added to the selected_data DataFrame: SIRS_Diagnosis and SEPSIS_Diagnosis.  The original regex-only method was used to populate these columns with 'Yes' or 'No' depending on whether the patient met the SIRS and Sepsis criteria.
- **Saving Final Dataset:** The final analytic dataset with the added diagnosis columns was saved to /content/drive/MyDrive/technicaltaskdata/final_analytic_dataset.csv.
- **Final Patient Counts:** Using the regex approach, the final dataset's total number of distinct patients diagnosed with SIRS or sepsis was tallied and printed.

**Reasoning:** Since no patients who met the requirements were found using the combined NER+Regex technique with the selected NER model, the final dataset was constructed using the findings from the original regex approach. The regex appraoch was further improved to account for different syntax combinations in the clinical notes. The results improved from a SIRS score of 67 to 594 and a SEPSIS diagnosis of 24 to 138.  This result implied that the regex patterns were more successful in this particular case at extracting the pertinent data from the dataset.  A clear 'Yes' or 'No' diagnosis column makes the dataset easily accessible for further modelling or analysis.  
